# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [37]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import xyzservices.providers as xyz

# Import API key
from api_keys import geoapify_key

In [39]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display
# Create a map that displays a point for every city with different colors based on the 'Country' column
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat', geo=True, size='Humidity', color='City',
    alpha=0.5, tiles='OSM', frame_width=800, frame_height=600,
    title='City Locations with Humidity Levels'
)

# Display the map
city_map

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
# Define ideal weather criteria
ideal_temp_min = 21   # Minimum ideal temperature in Celsius
ideal_temp_max = 33   # Maximum ideal temperature in Celsius
wind_speed_max = 4.5  # Maximum ideal wind speed

# Narrow down cities that fit the criteria
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= ideal_temp_min) &
    (city_data_df['Max Temp'] <= ideal_temp_max) &
    (city_data_df['Wind Speed'] <= wind_speed_max) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
print(f"Number of ideal cities: {len(ideal_cities_df)}")
ideal_cities_df.head()

Number of ideal cities: 21


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
81,makakilo city,US,21.3469,-158.0858,81,
152,kahului,US,20.8947,-156.4700,60,
197,gat,IL,31.6100,34.7642,100,
211,laguna,US,38.4210,-121.4238,79,
240,tikaitnagar,IN,26.9500,81.5833,59,
265,san quintin,MX,30.4833,-115.9500,74,
340,santa rosalia,MX,27.3167,-112.2833,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# I used this method because my API requests are taking too long so I looked for some examples, this is the result.

# This imports the ThreadPoolExecutor class from concurrent.futures module, 
# which is used for parallel execution of a function using multiple threads.
from concurrent.futures import ThreadPoolExecutor

hotel_df2= hotel_df
# Set parameters for Geoapify API
radius = 10000 
base_url = "https://api.geoapify.com/v2/places"
api_key = geoapify_key

# This function searches the nearest hotel for a given latitude and longitude using the Geoapify API.
# index: The row in the DataFrame.
def search_hotel(index, latitude, longitude):
# API request construction
    params = {
        "categories": "accommodation.hotel",
        "apiKey": api_key,
        "limit": 1,
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}"
    }
    response = requests.get(base_url, params=params)
    name_address = response.json()
# Error handling
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_name = "No hotel found"
# Returns a tuple
    return index, hotel_name

# This function updates the hotel_df2 DataFrame with the hotel name.
# Updates the "Hotel Name" column for the specified index in hotel_df2.
# Prints a message indicating the city and the nearest hotel found.
def update_hotel_df2(index, hotel_name):
    hotel_df2.loc[index, "Hotel Name"] = hotel_name
    print(f"{hotel_df2.loc[index, 'City']} - nearest hotel: {hotel_df2.loc[index, 'Hotel Name']}")

# Print a message to follow up the hotel search
print("Starting hotel search")

# EXECUTION BLOCK
# Purpose: Executes the hotel search in parallel for multiple locations.
# ThreadPoolExecutor is a method
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
# Iterates over each row in hotel_df2:
    for index, row in hotel_df2.iterrows():
# Retrieve the latitude and longitude from the row.
        latitude = row['Lat']
        longitude = row['Lng']
# The submit method takes the function search_hotel and its arguments (index, latitude, and longitude), 
# and schedules the function to run in a separate thread.
        futures.append(executor.submit(search_hotel, index, latitude, longitude))
    
    
    for future in futures:
# Retrieves the result (index and hotel name) from each future.
        index, hotel_name = future.result()
# Calls update_hotel_df2 to update the DataFrame with the hotel name.
        update_hotel_df2(index, hotel_name)

# Display sample data
hotel_df2.head()

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: Latitude20 Roebourne Village
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
lakki marwat - nearest hotel: No hotel found
tikrit - nearest hotel: فندق بلازا
guerrero negro - nearest hotel: Plaza sal paraiso
bilma - nearest hotel: No hotel found
pali - nearest hotel: Hotel Pawan International


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel


In [29]:
hotel_df2.to_csv("output_data/hotelsdata.csv", index=False)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Create a map that displays a point for every city with additional hover information
city_map = hotel_df2.hvplot.points(
    'Lng', 'Lat', geo=True, size='Humidity', color='Hotel Name', cmap='Category20',
    alpha=0.6, tiles='OSM', frame_width=800, frame_height=600,
    title='City Locations with Humidity Levels',
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
city_map

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.